In [1]:
import sys
import json
import torch
import inspect
# torch.set_printoptions(profile="full")

In [2]:
fq_init_params = {
    "torch.device": {"args": (), "kwargs": {"type": "cpu"}},
    "torch.Tensor": {"args": ([[1., 2., 3.]]), "kwargs": {}}
}

In [3]:
def full_name_with_qualname(klass: type) -> str:
    return f'{klass.__module__}.{klass.__qualname__}'

In [4]:
def detect_attrs(obj) -> dict:
    attr_dict = {"method":[], "function":[], "property":[], "builtin": []}
    for key in dir(obj):
        try:
            attr = getattr(obj, key)
            meth = inspect.ismethod(attr)
            if meth:
                # def(self) / @classmethod
                prop_type = "method"
            else:
                if inspect.isfunction(attr):
                    # @staticmethod / def / lambda
                    prop_type = "function"
                elif inspect.isbuiltin(attr):
                    # built-in
                    prop_type = "builtin"
                else:
                    prop_type = "property"
        except:
            # errors accessing an attr usually indicate a property executing
            prop_type = "property"

        attr_type = type(attr)

        type_name = full_name_with_qualname(attr_type)

        meth_dict = {"key": key, "type": type_name}
        if not type_name.startswith("builtins"):
            module_parts = type_name.split(".")
            klass = module_parts.pop()
            obj_type = getattr(sys.modules[".".join(module_parts)], klass)
            if type_name not in fq_init_params:
                print(f"Unknown init params for: {obj_type}")
                init_args = inspect.getfullargspec(obj_type.__init__)
                print(init_args)
            else:
                params = fq_init_params[type_name]
                o = obj_type(*params["args"], **params["kwargs"])
                meth_dict["children"] = o

        attr_dict[prop_type].append(meth_dict)
        
    return attr_dict

In [5]:
def print_attr(obj, obj_dict=None, level=0):
    padding = "".join(["\t"] * level)
    if obj_dict is None:
        obj_dict = detect_attrs(obj)

    print(f"{padding}{type(obj)} has:")
    for k, l in obj_dict.items():
        if len(l) > 0:
            print(f"\n{padding}{k}:")
            for d in l:
                k = d["key"]
                t = d["type"]
                if k.startswith("__"):
                    continue
                print(f"{padding}  -  {k} : {t}")
                if "children" in d:
                    print_attr(d["children"], level=level+1)

In [6]:
def compare_class_w_instance(klass, inst):
    a = set(list(dir(klass)))
    b = set(list(dir(inst)))

    print(a.difference(b))

In [7]:
# the thing we want to inspect
thing = torch.Generator
# thing = torch.Tensor

In [8]:
print(thing)
print(inspect.getfile(thing))

<class 'torch._C.Generator'>
/Users/madhavajay/.local/share/virtualenvs/PySyft-lHlz_cKe/lib/python3.6/site-packages/torch/_C.cpython-36m-darwin.so


In [9]:
inst = thing()
inst

In [10]:
print_attr(inst)

<class 'torch._C.Generator'> has:

property:
  -  _cdata : builtins.int
  -  device : torch.device
	<class 'torch.device'> has:

	property:
	  -  index : builtins.NoneType
	  -  type : builtins.str

	builtin:

builtin:
  -  get_state : builtins.builtin_function_or_method
  -  initial_seed : builtins.builtin_function_or_method
  -  manual_seed : builtins.builtin_function_or_method
  -  seed : builtins.builtin_function_or_method
  -  set_state : builtins.builtin_function_or_method


In [11]:
compare_class_w_instance(thing, inst)

set()


In [12]:
inspect.getfullargspec(torch.device.__init__)

FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [13]:
inspect.getclasstree([torch.Tensor, torch.nn.Parameter])

[(torch._C._TensorBase, (object,)),
 [(torch.Tensor, (torch._C._TensorBase,)),
  [(torch.nn.parameter.Parameter, (torch.Tensor,))]]]